In [3]:
import torch
import torch.nn as nn
import sentencepiece as spm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
sp = spm.SentencePieceProcessor()
sp.Load('tokenizer.model')  # Файл модели

True

In [5]:
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        
        # padding_idx=pad_idx говорит модели игнорировать нули (ускорение!)
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        
        # Параллельные свертки (для 2, 3 и 4 слов)
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embed_dim, 
                      out_channels=n_filters, 
                      kernel_size=fs)
            for fs in filter_sizes
        ])
        
        self.fc = nn.Linear(n_filters * len(filter_sizes), output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        # text: [batch, len] -> embedded: [batch, len, dim] -> [batch, dim, len]
        embedded = self.embedding(text).permute(0, 2, 1)
        
        # Свертка + ReLU + MaxPool для каждого размера фильтра
        conved = [F.relu(conv(embedded)) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        # Объединяем результаты
        cat = self.dropout(torch.cat(pooled, dim=1))
        return self.fc(cat)
    




VOCAB_SIZE = sp.GetPieceSize()
model = TextCNN(
    vocab_size=VOCAB_SIZE,
    embed_dim=300,
    n_filters=100,
    filter_sizes= [3,4,5],
    output_dim=3,
    dropout= 0.5,
    pad_idx=0
).to(device)


In [6]:
model.load_state_dict(torch.load('textcnn_best.pth'))

<All keys matched successfully>

In [7]:
torch.save(model, 'textcnn.pth')